<a href="https://colab.research.google.com/github/Ayric-cmad/initivate/blob/master/spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPARK 分析安徽各城市气象数据

In [ ]:
!apt install libkrb5-dev
!wget https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz
!tar xf spark-3.0.2-bin-hadoop3.2.tgz
!pip install findspark
!pip install pyspark
! pip install pyspark --user

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  comerr-dev krb5-multidev libgssrpc4 libkadm5clnt-mit11 libkadm5srv-mit11
  libkdb5-9
Suggested packages:
  doc-base krb5-doc krb5-user
The following NEW packages will be installed:
  comerr-dev krb5-multidev libgssrpc4 libkadm5clnt-mit11 libkadm5srv-mit11
  libkdb5-9 libkrb5-dev
0 upgraded, 7 newly installed, 0 to remove and 45 not upgraded.
Need to get 349 kB of archives.
After this operation, 1,992 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgssrpc4 amd64 1.16-2ubuntu0.2 [54.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libkdb5-9 amd64 1.16-2ubuntu0.2 [37.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates

In [ ]:
!pip install pyecharts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.3 MB/s 
     |████████████████████████████████| 130 kB 44.2 MB/s 


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()


In [ ]:
data = [1,2,3,4,5]
distData = sc.parallelize(data,8)
s = distData.reduce(lambda a,b: a*b)
print(s)

120


In [ ]:
file = ["line1","This is line2"]
lines = sc.parallelize(file)
print(lines.collect())
lineLengths = lines.map(lambda line: len(line))

print(lineLengths.collect())
totalLength = lineLengths.reduce(lambda a,b: a+b)
print(totalLength)
lineLengths.persist()
print(lineLengths.collect())

['line1', 'This is line2']
[5, 13]
18
[5, 13]


# 爬虫

In [ ]:
import pandas as pd
import numpy as np
import re
import requests
from lxml import etree
import time

# ah_citys = ["合肥","淮北","亳州","宿州","蚌埠","阜阳","淮南","滁州","六安","马鞍山","芜湖","宣城","铜陵","池州","安庆","黄山"]
ah_citys = ["池州","安庆","黄山"]
#ah_citys_pinyin = ["hefei","huaibei","bozhou","anhuisuzhou","bangbu","fuyang","huainan","chuzhou","liuan","maanshan","wuhu","xuancheng","tongling","chizhou","anqing","huangshan"]
ah_citys_pinyin = ["chizhou","anqing","huangshan"]
years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]
months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
# url = "http://www.tianqihoubao.com/lishi/gdsuixi/month/202204.html"

header = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.62 Safari/537.36",
    }


dataset = pd.DataFrame()
for i,city in enumerate(ah_citys_pinyin):
    if(city == "chizhou"):
        for year in ["2019","2020","2021"]:
            for month in months:
                url = "http://www.tianqihoubao.com/lishi/" + city + "/month/" + year + month + ".html"
                r = requests.get(url=url,headers = header)
                print(url)
                # time.sleep(5)
                df = pd.read_html(r.text,header = 0)[0]
                df["城市"] = ah_citys[i]
                dataset = pd.concat([dataset,df])
    else:
        for year in years:
            for month in months:
                url = "http://www.tianqihoubao.com/lishi/" + city + "/month/" + year + month + ".html"
                r = requests.get(url=url,headers = header)
                print(url)
                # time.sleep(5)
                df = pd.read_html(r.text,header = 0)[0]
                df["城市"] = ah_citys[i]
                dataset = pd.concat([dataset,df])
dataset.to_csv("qixiangdataset.csv",encoding="utf_8_sig",index = False)

# 数据预处理

In [ ]:
from pyspark.sql import Row
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

import pandas as pd
from google.colab import files

spark=SparkSession.builder.appName("lz").config("spark.driver.memory", "2g").getOrCreate()
sc = SparkContext.getOrCreate()

#files.upload()
dataset_rdd = sc.textFile("qixiangdataset.csv",use_unicode='utf-8')
header = dataset_rdd.first()
dataset_rdd = dataset_rdd.filter(lambda row:row != header)
dataset_rdd = dataset_rdd.map(lambda line:line.split(","))
dataset_df = dataset_rdd.map(lambda line: Row (date = line[0],
                   tq=line[1],
                   temp=line[2],
                   wind=line[3],
                   city=line[4])).toDF()

dataset_df.show(5)

+------------+-----------+----------+-------------------------+----+
|        date|         tq|      temp|                     wind|city|
+------------+-----------+----------+-------------------------+----+
|2011年1月1日|  多云  /阴|4℃  /  -3℃| 东北风 微风  /北风 3-4级|合肥|
|2011年1月2日|小雪  /小雪|2℃  /  -2℃|东北风 3-4级  /北风 3-4级|合肥|
|2011年1月3日|  阴  /多云|3℃  /  -4℃|  北风 3-4级  /北风 3-4级|合肥|
|2011年1月4日|    晴  /晴|4℃  /  -3℃|  北风 3-4级  /西风 3-4级|合肥|
|2011年1月5日|    阴  /阴|5℃  /  -3℃|西北风 3-4级  /北风 3-4级|合肥|
+------------+-----------+----------+-------------------------+----+
only showing top 5 rows



In [ ]:
#处理单元格内的数据

from pyspark.sql.functions import * 

#处理特殊字符
dataset_df = dataset_df.withColumn('date',regexp_replace(dataset_df['date'],"年","-"))
dataset_df = dataset_df.withColumn('date',regexp_replace(dataset_df['date'],"月","-"))
dataset_df = dataset_df.withColumn('date',regexp_replace(dataset_df['date'],"日",""))
dataset_df = dataset_df.withColumn('temp',regexp_replace(dataset_df['temp'],"℃",""))

#处理日期
from pyspark.sql.types import DateType
dataset_df = dataset_df.withColumn('date', dataset_df['date'].cast(DateType()))

#分割气温并且去除空格
dataset_df = dataset_df.withColumn("最高气温", trim(split("temp", "\/")[0]))\
        .withColumn("最低气温", trim(split("temp", "\/")[1]))

dataset_df = dataset_df.withColumn("白天天气", trim(split("tq", "\/")[0]))\
        .withColumn("夜间天气", trim(split("tq", "\/")[1]))

dataset_df = dataset_df.withColumn("白天风向风力", trim(split("wind", "\/")[0]))\
        .withColumn("夜间风向风力", trim(split("wind", "\/")[1]))

dataset_df = dataset_df.drop('temp','tq','wind')

dataset_df.show(5)

+----------+----+--------+--------+--------+--------+------------+------------+
|      date|city|最高气温|最低气温|白天天气|夜间天气|白天风向风力|夜间风向风力|
+----------+----+--------+--------+--------+--------+------------+------------+
|2011-01-01|合肥|       4|      -3|    多云|      阴| 东北风 微风|  北风 3-4级|
|2011-01-02|合肥|       2|      -2|    小雪|    小雪|东北风 3-4级|  北风 3-4级|
|2011-01-03|合肥|       3|      -4|      阴|    多云|  北风 3-4级|  北风 3-4级|
|2011-01-04|合肥|       4|      -3|      晴|      晴|  北风 3-4级|  西风 3-4级|
|2011-01-05|合肥|       5|      -3|      阴|      阴|西北风 3-4级|  北风 3-4级|
+----------+----+--------+--------+--------+--------+------------+------------+
only showing top 5 rows




# 可视化

In [ ]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.NTERACT

import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Bar, Line, Pie

ah_citys = ["合肥","淮北","亳州","宿州","蚌埠","阜阳","淮南","滁州","六安","马鞍山","芜湖","宣城","铜陵","池州","安庆","黄山"]

## 五年合肥气温变化

In [ ]:
import pyspark.sql.functions as f
import numpy as np
#添加年份列
df_year = dataset_df.withColumn("年份",f.year(f.to_timestamp('date',"yyyy/MM/dd")))
df_year.show(5)

#处理空值
df_year = df_year.filter(df_year["最高气温"] != '')
df_year = df_year.filter(df_year["最低气温"] != '')
df_year.show(5)

df_five = df_year.filter(df_year["年份"] >= 2017).filter(df_year["city"] == "合肥")
df_avg = df_five.withColumn("平均气温",(df_five["最高气温"]+df_five["最低气温"])/2)
df_avg.show(5)

fiveyear = np.ravel(df_avg.select("date").collect())
tempmax = np.ravel(df_avg.select("最高气温").collect())
tempmin = np.ravel(df_avg.select("最低气温").collect())
tempavg = np.ravel(df_avg.select("平均气温").collect())

+----------+----+--------+--------+--------+--------+------------+------------+----+
|      date|city|最高气温|最低气温|白天天气|夜间天气|白天风向风力|夜间风向风力|年份|
+----------+----+--------+--------+--------+--------+------------+------------+----+
|2011-01-01|合肥|       4|      -3|    多云|      阴| 东北风 微风|  北风 3-4级|2011|
|2011-01-02|合肥|       2|      -2|    小雪|    小雪|东北风 3-4级|  北风 3-4级|2011|
|2011-01-03|合肥|       3|      -4|      阴|    多云|  北风 3-4级|  北风 3-4级|2011|
|2011-01-04|合肥|       4|      -3|      晴|      晴|  北风 3-4级|  西风 3-4级|2011|
|2011-01-05|合肥|       5|      -3|      阴|      阴|西北风 3-4级|  北风 3-4级|2011|
+----------+----+--------+--------+--------+--------+------------+------------+----+
only showing top 5 rows

+----------+----+--------+--------+--------+--------+------------+------------+----+
|      date|city|最高气温|最低气温|白天天气|夜间天气|白天风向风力|夜间风向风力|年份|
+----------+----+--------+--------+--------+--------+------------+------------+----+
|2011-01-01|合肥|       4|      -3|    多云|      阴| 东北风 微风|  北风 3-4级|2011|
|

In [ ]:
fiveyear.shape

(1832,)

In [321]:
from pyecharts.options.series_options import LineStyleOpts
from pyecharts.types import MarkPoint
opt = opts.InitOpts(theme=ThemeType.DARK,width = "1000px",height="400px")
#opt = opts.InitOpts(theme=ThemeType.INFOGRAPHIC)
chart = (
    Line(init_opts=opt)#设置主题样式
    .add_xaxis([str(w) for w in fiveyear][::10])
    .add_yaxis("合肥最高气温",tempmax[::10],is_smooth=True,is_symbol_show = False,markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max')]))
    .add_yaxis("合肥最低气温",tempmin[::10],is_smooth=True,is_symbol_show = False,markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min')]))
    .add_yaxis("合肥平均气温",tempavg[::10],is_smooth=True,is_symbol_show = False)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="合肥五年气温变化"),#设置标题
        yaxis_opts=opts.AxisOpts(min_=-20,max_=50),#设置坐标轴范围
        )
    .set_series_opts(linestyle_opts=LineStyleOpts(width=2))#修改线宽
)
chart.load_javascript()
chart.render("合肥气温变化.html")
chart.render_notebook()

## 降水天气最多的城市

In [ ]:
dataset_df.show(5)

+----------+----+--------+--------+--------+--------+------------+------------+
|      date|city|最高气温|最低气温|白天天气|夜间天气|白天风向风力|夜间风向风力|
+----------+----+--------+--------+--------+--------+------------+------------+
|2011-01-01|合肥|       4|      -3|    多云|      阴| 东北风 微风|  北风 3-4级|
|2011-01-02|合肥|       2|      -2|    小雪|    小雪|东北风 3-4级|  北风 3-4级|
|2011-01-03|合肥|       3|      -4|      阴|    多云|  北风 3-4级|  北风 3-4级|
|2011-01-04|合肥|       4|      -3|      晴|      晴|  北风 3-4级|  西风 3-4级|
|2011-01-05|合肥|       5|      -3|      阴|      阴|西北风 3-4级|  北风 3-4级|
+----------+----+--------+--------+--------+--------+------------+------------+
only showing top 5 rows



In [ ]:
import pyspark.sql.functions as f
import numpy as np

#处理出下雨的数据
expression = ".*雨.*|.*雪.*"
df_rain = dataset_df.filter(dataset_df['白天天气'].rlike(expression)|dataset_df['夜间天气'].rlike(expression))#筛选出天气中含有雨或雪的天气
df_rain.show(10)
df_rain = df_rain.groupby("city").count()
df_rain.show(10)
df_rain_city = np.ravel(df_rain.select("city").collect())
df_rain_count = np.ravel(df_rain.select("count").collect())

+----------+----+--------+--------+--------+--------+------------+------------+
|      date|city|最高气温|最低气温|白天天气|夜间天气|白天风向风力|夜间风向风力|
+----------+----+--------+--------+--------+--------+------------+------------+
|2011-01-02|合肥|       2|      -2|    小雪|    小雪|东北风 3-4级|  北风 3-4级|
|2011-01-17|合肥|       3|      -3|    多云|    小雪|东北风 3-4级|东北风 3-4级|
|2011-01-18|合肥|       2|      -3|    小雪|    小雪|东北风 3-4级|东北风 3-4级|
|2011-01-19|合肥|       2|      -2|      阴|    小雪|东北风 3-4级|   东风 微风|
|2011-01-20|合肥|       1|      -2|    小雪|      阴|   东风 微风|   东风 微风|
|2011-01-27|合肥|       4|      -1|  雨夹雪|    中雪|  东风 3-4级|东北风 3-4级|
|2011-01-28|合肥|       2|      -3|    小雪|      阴|东北风 3-4级|西北风 3-4级|
|2011-02-08|合肥|      17|       5|    多云|    小雨| 东北风 微风|东北风 3-4级|
|2011-02-09|合肥|       7|       0|    小雨|  雨夹雪|东北风 3-4级|东北风 4-5级|
|2011-02-10|合肥|       2|      -1|  雨夹雪|      阴|东北风 4-5级|东北风 3-4级|
+----------+----+--------+--------+--------+--------+------------+------------+
only showing top 10 rows

+----+-----+
|city|c

In [ ]:
sumday = df_rain_count.sum()
raindaylist = [w/sumday for w in df_rain_count]
[list(z) for z in zip(df_rain_city,raindaylist)]

[['阜阳', 0.051741544674406864],
 ['淮北', 0.045053003533568906],
 ['宿州', 0.048334174659263],
 ['淮南', 0.05439172135285209],
 ['合肥', 0.06179538953390543],
 ['六安', 0.06604408547871446],
 ['滁州', 0.06032306915699142],
 ['亳州', 0.04534746760895171],
 ['蚌埠', 0.05157327948847384],
 ['铜陵', 0.07218576476527007],
 ['黄山', 0.08560491334342925],
 ['池州', 0.07424701329294969],
 ['宣城', 0.07576139996634697],
 ['马鞍山', 0.06650681474003028],
 ['芜湖', 0.06882046104660945],
 ['安庆', 0.07226989735823658]]

In [ ]:
chart_pie = (
    Pie()
    .add("降水天气最多的城市",[list(z) for z in zip(df_rain_city,raindaylist)],center=["40%", "50%"])
)
chart_pie.load_javascript()
chart_pie.render("降水最多城市.html")
chart_pie.render_notebook()

In [ ]:
df_rain_city

array(['阜阳', '淮北', '宿州', '淮南', '合肥', '六安', '滁州', '亳州', '蚌埠', '铜陵', '黄山',
       '池州', '宣城', '马鞍山', '芜湖', '安庆'], dtype='<U3')

In [322]:
from pyecharts.charts import Map
from pyecharts import options as opts
from pyecharts.globals import ThemeType

value = list(df_rain_count/100)
attr = ['阜阳市','淮北市','宿州市','淮南市','合肥市','六安市','滁州市','亳州市','蚌埠市','铜陵市','黄山市','池州市','宣城市','马鞍山市','芜湖市','安庆市']
sequence = list(zip(attr,value))

def map_visualmap(sequence,year) -> Map:
    c = (
        Map()
        .add(year,sequence, "安徽",)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="地图"),
            visualmap_opts=opts.VisualMapOpts(max_=21,min_=10),
#             visualmap_opts=opts.VisualMapOpts(max_=101,range_color=["lightskyblue", "yellow", "orangered"]),
        )
    )
    return c

map = map_visualmap(sequence,'下雨天气')
map.render("下雨天气热力图.html")
map.render_notebook()